In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import napari
from scribbles_testing.FoodSeg103_data_handler import *

c:\Users\roman\anaconda3\envs\seghub_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prediction

Load the images as batches

In [2]:
img_nums = [1328]#[n for n in range(0, 4500, 1000)] #2750 #1234 #2314
imgs = load_food_batch(img_nums, load_gts=False)[0]
num_imgs = len(imgs)
print(f"Total images: {num_imgs}")

Total images: 1


Check and filter for resolution

In [3]:
resol = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}

# Check for images with resolution over a certain threshold
thresh = 640_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
1328:    90,000 pixels


In [4]:
# Filter out those images
imgs = {im_num: im for im_num, im in imgs.items() if resol[im_num] <= thresh}
num_imgs = len(imgs)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(im.shape[:2]) for im_num, im in imgs.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 640,000: 1
1328:  90,000 pixels


In [22]:
# Where to find and save the data
folder_path = "../FoodSeg103_results/data/food_run06a_examples"

# Which scribbles to use
mode = "all"
bins = [0.1] #[0.05, 0.3, 1.5]
scribble_width = 2
# suff = ["TEST"]
s = "run06"

# Which prediction models to use
convpaint_layers = [[0], [0,2], [0,2,5]]
convpaint_scalings = [[1], [1,2], [1,2,4], [1,2,4,8]]
model = "vgg16" # 'vgg16', 'efficient_netb0', 'single_layer_vgg16', 'single_layer_vgg16_rgb', 'dino_vits16'
use_dino = True
use_ilastik = True
pred_seed = 123

save_res = False
show_res = False

Loop and predict

In [24]:
for bin in bins:
    for img_num, img in imgs.items():
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        for layers in [[0]]:#convpaint_layers:
            for scalings in [[1,2]]:#convpaint_scalings:
                print(f"   ConvPaint: layers {layers}, scalings {scalings}")
                pred_conv = pred_food_convpaint(img, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, 
                                                layer_list=layers, scalings=scalings, model=model, random_state=12,
                                                save_res=save_res, show_res=show_res, ground_truth=None)
        if use_ilastik:
            print("   Ilastik")
            pred_ila = pred_food_ilastik(img, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s,
                                         random_state=pred_seed,
                                         save_res=save_res, show_res=show_res, ground_truth=None)
        if use_dino:
            print("   DINO")
            pred_dino = pred_food_dino(img, folder_path, img_num, mode=mode, bin=bin, scribble_width=scribble_width, suff=s, 
                                            dinov2_model='s_r', upscale_order=0, random_state=pred_seed,
                                            save_res=save_res, show_res=show_res, ground_truth=None)
        print("\n")

IMG 1328: bin 0.1, suff run06
   ConvPaint: layers [0], scalings [1, 2]
   Ilastik
   DINO




Show the last predictions (optionally with image, ground truth and scribbles)

In [7]:
ground_truth = load_food_data(img_num, load_image=False)[1]
scribbles = get_food_img_data(folder_path, img_num, load_scribbles=True, mode=mode, bin=bin, scribble_width=scribble_width, suff=s)["scribbles"]

In [25]:
print(f"Ilastik:   {np.sum(pred_ila == ground_truth) / np.prod(pred_ila.shape) * 100:.2f}%")
print(f"Convpaint: {np.sum(pred_conv == ground_truth) / np.prod(pred_conv.shape) * 100:.2f}%")
print(f"DINOv2:    {np.sum(pred_dino == ground_truth) / np.prod(pred_dino.shape) * 100:.2f}%")

Ilastik:   64.99%
Convpaint: 72.79%
DINOv2:    93.19%


In [ ]:
v = napari.Viewer()
v.add_image(img)
v.add_labels(ground_truth)
v.add_labels(scribbles)
v.add_labels(pred_conv)
v.add_labels(pred_ila)
v.add_labels(pred_dino)